In [ ]:
import numpy as np
import optuna
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
np.random.seed(16)

# prior
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
canonical_0 = np.array([[1.0], [1.0]])
covm_0 = np.linalg.inv(precision_0)
mean_0 = covm_0 @ canonical_0

precisions_0 = [precision_0]
canonicals_0 = [canonical_0]
logcons_0 = [np.linalg.slogdet(precision_0)[1] / 2 - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item() / 2]
pit_0 = [0]

# true value
alpha, beta = np.random.multivariate_normal(mean_0.flatten(), covm_0)
print(alpha, beta, myopic(canonical_0, precision_0))

In [ ]:
def objective(trial):
    p = trial.suggest_float('p', 0.0, 0.1)
    loglike = marginal_likelihood(p, canonicals_0, precisions_0, logcons_0, pit_0, xs, ys)
    return loglike

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

In [ ]:
study.best_trial

In [ ]:
def objective_change(trial):
    a = trial.suggest_float('a', -10.0, 10.0)
    q = q_myopic_with_change(canonicals_0, precisions_0, logcons_0, pit_0, a, alpha, beta, mean_0, covm_0, p=0.05)
    return q

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_change, n_trials=10)

In [ ]:
study.best_trial

In [ ]:
xs = np.random.uniform(-5.0, 5.0, 100)
ys = []
for i in range(100):
    if i == 100:
        alpha, beta = np.random.multivariate_normal(mean_0.flatten(), covm_0)
        print(alpha, beta)
    ys.append(env_response(xs[i], alpha, beta)[0])
ys = np.array(ys)